# Configurar un SwapICPCLP de Mercado

Se muestra una forma posible de configurar una operación de Swap ICPCLP de mercado. Se definirán algunos parámetros por default y sólo será necesario especificar el nocional, el plazo, el valor de la tasa fija y si ésta se recibe o se paga para obtener la estructura completa.

Para ejecutar todos los ejemplos se debe importar la librería. Se sugiere utilizar siempre el alias `qcf`. 

In [ ]:
import qcfinancial as qcf
import aux_functions as aux

Se verifica la versión y build de `qcfinancial`.

## Parámetros por Default de la Operación

Se comienza estableciendo una fecha de trade y dando de alta un `dict` con los parámetros por default comunes y los específicos de ambas patas.

In [ ]:
trade_date = qcf.QCDate(14, 6, 2024)

In [ ]:
both_default_values = {
    "bus_adj_rule": qcf.BusyAdjRules.MODFOLLOW,
    "settlement_calendar": qcf.BusinessCalendar(trade_date, 20),
    "settlement_stub_period": qcf.StubPeriod.NO,
    "settlement_lag": 1,
    "amort_is_cashflow": False,
    "notional_currency": qcf.QCCLP(),
    "sett_lag_behaviour": qcf.SettLagBehaviour.DONT_MOVE
}

icpclp_default_values = {
    "fix_adj_rule": qcf.BusyAdjRules.MODFOLLOW,
    "fixing_calendar": qcf.BusinessCalendar(trade_date, 20),
    "dates_for_eq_rate": qcf.DatesForEquivalentRate.ACCRUAL,
    "interest_rate": qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf()),
    "eq_rate_decimal_places": 4,
}

fixed_rate_default_values = {
    "is_bond":False,  
}

## Pata Fija

Se da de alta los parámetros variables de la pata fija. Notar la utilización del parámetro auxiliar `maturity` que permite calcular la fecha final.

In [ ]:
str_maturity = "1Y"
maturity = qcf.Tenor(str_maturity)
meses = maturity.get_months() + 12 * maturity.get_years()

Se da de alta el objeto.

In [ ]:
fixed_rate_value = .03
fixed_rate_leg_other_values = {
    "settlement_periodicity": qcf.Tenor('2Y') if str_maturity in ['1M', '2M', '3M', '6M', '9M', '12M', '1Y', '18M'] else qcf.Tenor('6M'),
    "rec_pay": qcf.RecPay.RECEIVE,
    "initial_notional": 1_000_000_000.0,
    "start_date": (start_date:=qcf.QCDate(18, 6, 2024)),
    "end_date": start_date.add_months(meses),
    "interest_rate": qcf.QCInterestRate(fixed_rate_value, qcf.QCAct360(), qcf.QCLinearWf()),  
}

In [ ]:
fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    **(both_default_values | fixed_rate_default_values | fixed_rate_leg_other_values)
)

Se visualiza el resultado.

In [ ]:
aux.leg_as_dataframe(fixed_rate_leg).style.format(aux.format_dict)

## Pata OvernightIndex

Se da de alta los parámetros variables de la pata OvernightIndex. Notar que se reutilizan algunos de los parámetros de la pata fija.

In [ ]:
rp = qcf.RecPay.RECEIVE if fixed_rate_leg_other_values['rec_pay'] == qcf.RecPay.PAY else qcf.RecPay.RECEIVE

icpclp_leg_other_values = {
    "rec_pay": rp,
    "initial_notional": fixed_rate_leg_other_values["initial_notional"],
    "start_date": fixed_rate_leg_other_values["start_date"],
    "end_date": fixed_rate_leg_other_values["end_date"],
    "settlement_periodicity": fixed_rate_leg_other_values["settlement_periodicity"],
    "interest_rate": qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    "index_name": "ICPCLP",
    "spread": 0.0,
    "gearing": 1.0,
}

Finalmente, se da de alta el objeto.

In [ ]:
on_index_leg = qcf.LegFactory.build_bullet_overnight_index_leg(
    **(both_default_values | icpclp_default_values | icpclp_leg_other_values)
)

Se visualiza.

In [ ]:
aux.leg_as_dataframe(on_index_leg).style.format(aux.format_dict)